## QJE Tesseract Extraction

In [22]:
import fitz

# for OCR using PyTesseract
import cv2                              # pre-processing images
import pytesseract                      # extracting text from images
import numpy as np
import matplotlib.pyplot as plt         # displaying output images
from PIL import Image
import regex
import pandas as pd
import time
import os

In [23]:
zoom_x = 1.0 # horizontal zoom
zoom_y = 1.0 # vertical zoom
mat = fitz.Matrix(zoom_x, zoom_y)

In [24]:
base_path="/Users/sijiawu/Work/Refs Danae/Thesis/Data"
temp=base_path+'/PDFs/QJE/'

In [25]:
Merged=pd.read_excel(base_path+'/Combined/QJE_M_sco_du.xlsx')
Merged.loc[Merged['journal']=="The Quarterly Journal of Economics",'journal']='QJE'

In [1]:
investigate=Merged[(Merged.year<=1980)&(Merged.year>=1970)&(Merged.content_type!="MISC")&(Merged.content_type!="Review")]


NameError: name 'Merged' is not defined

In [ ]:
for i in investigate.index:
    filepath=base_path+'/PDFs/QJE/'+Merged.iloc[i]['URL'].split('/')[-1]+'.pdf'
    if os.path.exists(filepath):
        doc=fitz.open(filepath)
        print(Merged.iloc[i]['year'])
        print(Merged.iloc[i]['number'])
        print(Merged.iloc[i]['volume'])
        print(Merged.iloc[i]['author'])
        print(Merged.iloc[i]['title'])
#         for page in doc:
            #if (page.number == 1) or (page.number==(doc.page_count-1)):
        png = base_path+"/PDFs/QJE/png/" + Merged.iloc[i]['URL'].split('/')[-1].split('.')[0] + '_wo_cover_page-0.png'
        print(os.path.exists(png))
        print(png)
        if os.path.exists(png)==True:
            original_image = cv2.imread(png)

            # convert the image to grayscale
            gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

            plt.figure(figsize=(25, 15))
            plt.imshow(gray_image, cmap='gray')
            plt.show()

In [ ]:
import string
def converter(teststring):
    teststring=teststring.replace('.','[.]')
    print(teststring)
    #for i in range(len(teststring)):
    #    if teststring[i] == '.':
    #        teststring=teststring[0:i-1]+'.'+teststring[i:]
    #teststring=teststring.replace('I','.')
            
    for i in teststring:
        if (i not in string.ascii_lowercase) & (i not in string.ascii_uppercase) & (i !=' ') & (i!='.') & (i!='[') & (i!=']'):
            teststring=teststring.replace(i,'.')
    return teststring


In [ ]:
def getNumberofPages(text):
    if pd.isna(text)==False:
        if re.search('\d',text):
            temp=text.split(',')
            pages=0
            print(temp)
            for m in temp:
                if '-' in m:
                    t=str(m).split('-')
                    pages=pages+int(re.sub('\D','',t[1]))-int(re.sub('\D','',t[0]))+1
                else:
                    pages+=1
            return pages
    return 1



In [ ]:
def generate_refs(SCANNED_FILE, mat, path, k_val, keyword):
    doc = fitz.open(SCANNED_FILE)
    parsed={}
    references={}
    found=0
    for page in reversed(doc):
        if (page.number >= 1):
            png = path+"\\" + SCANNED_FILE.split('\\')[-1].split('.')[0] + '_page-%i.png' % page.number
            if os.path.exists(png)==False:
                pix = page.get_pixmap(matrix=mat)
                print(png)
                pix.save(png)

            parsed[page.number]=[]
            references[page.number]=[]

            original_image = cv2.imread(png)
            # convert the image to grayscale
            gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(gray_image, cmap='gray')
            #plt.show()

            # Performing OTSU threshold
            ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(threshold_image, cmap='gray')
            #plt.show()

            rectangular_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (k_val, k_val))

            # Applying dilation on the threshold image
            dilated_image = cv2.dilate(threshold_image, rectangular_kernel, iterations = 1)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(dilated_image)
            #plt.show()

            # Finding contours
            contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            # Creating a copy of the image
            copied_image = original_image.copy()

            mask = np.zeros(original_image.shape, np.uint8)
            #i=1
            # Looping through the identified contours
            # Then rectangular part is cropped and passed on to pytesseract
            # pytesseract extracts the text inside each contours
            # Extracted text is then written into a text file
            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                # Cropping the text block for giving input to OCR
                cropped = copied_image[y:y + h, x:x + w]
                # Apply OCR on the cropped image
                text = pytesseract.image_to_string(cropped, lang='lat', config='--oem 3 --psm 4')
                #print(i)
                print(text)
                parsed[page.number].append(text)
                print(regex.search(keyword,text.upper()))
                if regex.search(keyword, text.upper()) is not None:
                    print('found')
                    return {'found': parsed}
                #masked = cv2.drawContours(mask, [cnt], 0, (255, 255, 255), -1)
                #i=i+1
    return {'raw': parsed}

In [ ]:
def generate_pngs(SCANNED_FILE, mat, path, k_val, keyword, ignore):
    doc = fitz.open(SCANNED_FILE)
    parsed={}
    references={}
    found=0
    for page in reversed(doc):
        if (page.number >= 1):
            png = path+"\\" + SCANNED_FILE.split('\\')[-1].split('.')[0] + '_page-%i.png' % page.number
            if os.path.exists(png)==False:
                pix = page.get_pixmap(matrix=mat)
                print(png)
                pix.save(png)

            parsed[page.number]=[]
            references[page.number]=[]

            original_image = cv2.imread(png)
            # convert the image to grayscale
            gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(gray_image, cmap='gray')
            #plt.show()

            # Performing OTSU threshold
            ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(threshold_image, cmap='gray')
            #plt.show()

            rectangular_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (k_val, k_val))

            # Applying dilation on the threshold image
            dilated_image = cv2.dilate(threshold_image, rectangular_kernel, iterations = 1)

            #plt.figure(figsize=(25, 15))
            #plt.imshow(dilated_image)
            #plt.show()

            # Finding contours
            contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            # Creating a copy of the image
            copied_image = original_image.copy()

            mask = np.zeros(original_image.shape, np.uint8)
            #i=1
            # Looping through the identified contours
            # Then rectangular part is cropped and passed on to pytesseract
            # pytesseract extracts the text inside each contours
            # Extracted text is then written into a text file
            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                # Cropping the text block for giving input to OCR
                cropped = copied_image[y:y + h, x:x + w]
                # Apply OCR on the cropped image
                text = pytesseract.image_to_string(cropped, lang='lat', config='--oem 3 --psm 4')
                #print(i) 
                #print(regex.search(keyword,text.upper()))
                if text=='':
                    #print('empty af')
                    continue
                
                parsed[page.number].append(text)    
                
                if regex.search(ignore, text.upper()) is not None:
                    print('000110000')
                    #print(regex.search(ignore, text.upper()))
                    #print(text)
                    continue
                
                if regex.search('[a-zA-Z]', text.upper()) is None:
                    print('00020000')
                    #print(text)
                    continue
                    
                print(text)
                
                if found ==1:
                    print('found')
                    if regex.search('[a-zA-Z]', text.upper()) is not None:
                        
                        return {'found':text,'raw': parsed}
                if regex.search(keyword, text.upper()) is not None:
                    print('found')
                    found=1
                
                #masked = cv2.drawContours(mask, [cnt], 0, (255, 255, 255), -1)
                #i=i+1
    return {'raw': parsed}

In [ ]:

SCANNED_FILE=path+'\\40961018.pdf'
SCANNED_FILE=path+'\\25098840.pdf'
SCANNED_FILE=path+'\\27867508.pdf'
#SCANNED_FILE=path+'\\2586940.pdf'
#SCANNED_FILE=path+'\\2586895.pdf'
#SCANNED_FILE=path+'\\2696451.pdf'
SCANNED_FILE=path+'\\40961014.pdf'
SCANNED_FILE=path+'\\2951257.pdf'
t0=time.time()
ignore='((ARE GHETTOS GOOD OR BAD?){e<=3}(\n| \d)|^(THIS CONTENT DOWNLOADED){e<=3}|^(QUARTERLY JOURNAL OF ECONOMICS){e<=3}\n|^\d)'
affiliations=generate_pngs(SCANNED_FILE, mat, path, 30, '(^|\n)(REFERENCES){e<=3}(\n| )', ignore)
t1=time.time()
total=t1-t0
print(total)
affiliations

In [47]:
# This version ignores contouring and sectioning out paragraphs. It directly feeds the image to tesseract. 
# There seems to be no image resolution degradation this way as opposed to reduced using the openCV library.
# SCANNED_FILE: is for the full path to the original pdf. we require this to get the number of pages. 
#  The assumption is that the jstor (or other) cover page has been removed previously, so in our case it will always have wo_cover.pdf as suffix.
# path: to the folder containing pre-generated pngs, the pngs in this folder are assumed to have the same file name as the SCANNED_FILE + suffix page-{page no}.png for each sharded page
# keyword: this is whatever regex pattern that you wish to search for. This function uses the regex.search method from the regex library
#  It can take fuzzy match regex patterns
# config: this is the tesseract configuration default is '--oem 1 --psm 3', which is also the default for this function
#  3 implies automatic page segmentation, better for 2 column format pdfs, 6 assumes single column, top to bottom text and will preserve each line ending better.
def generate_refs2(SCANNED_FILE, path, keyword, custom_config = r'--oem 1 --psm 3'):
    try:
        doc = fitz.open(SCANNED_FILE)
    except:
        print("could not open: "+SCANNED_FILE)
        raise Exception("this file is corrupt")
    if "wo_cover" not in SCANNED_FILE:
        print("warning, the file: "+SCANNED_FILE.split('/')[-1]+" does not have it's coverpage removed.\nThis function will continue. Assumed image file name convention is: "+SCANNED_FILE.split('/')[-1].split('.')[0] + '_page-{number}.png')
    parsed={}
    references={}
    found=0
    for page in reversed(doc):
        png = path+"/" + SCANNED_FILE.split('/')[-1].split('.')[0] + '_page-%i.png' % page.number
#             print(png)
        parsed[page.number]=[]
        references[page.number]=[]
        if os.path.exists(png)==True:
            text = pytesseract.image_to_string(png, config=custom_config)
#                 print(text)
            parsed[page.number].append(text)
            keyword_search=regex.search(keyword,text.upper())
            if keyword_search is not None:
                print('found')
                return [{'found': parsed}, {"pages":doc.page_count}]
        else:
            print("error: this image does not exist, please generate png shards at 300 dpi in path: "+path)
    print("the keyword: "+keyword + "was not found. But this is the full tesseract output nonetheless.")
    return {'raw': parsed}

## investigative notes

2017 and previous papers are from JSTOR. You may us single column top to bottom config without page segmentation. Post 2017 papers req psm 3 because the watermark is vertical and on the right side of the page.

In [62]:
t0=time.time()
custom_config = r'--oem 1 --psm 3'

for i in Merged[(Merged['year']<=2020) & (Merged['year']>2017)& (Merged['content_type']!='MISC') & (Merged['content_type']!='Review')].index:
    filepath=temp+'wo_cover/'+Merged.iloc[i]['URL'].split('/')[-1]+'_wo_cover.pdf'
    print(filepath)
    if os.path.exists(filepath)==True:
        print(Merged.iloc[i][['year','number','volume', 'URL']])
        references=generate_refs2(filepath, temp+"png", '(^|\n)R(EFERENCES){e<=3}(\n| )', custom_config)
        dict_ref[Merged.iloc[i]['URL'].split('/')[-1]]={'references':references, 'URL': Merged.iloc[i]['URL']}
    else:
        dict_ref[Merged.iloc[i]['URL'].split('/')[-1]]='PDF not available, download at '+ Merged.iloc[i]['URL']
t1=time.time()
total=t1-t0
print(total)
print(i)

/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjaa024_wo_cover.pdf
year                                     2020
number                                      4
volume                                    135
URL       https://doi.org/10.1093/qje/qjaa024
Name: 0, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjaa022_wo_cover.pdf
year                                     2020
number                                      4
volume                                    135
URL       https://doi.org/10.1093/qje/qjaa022
Name: 1, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjaa021_wo_cover.pdf
year                                     2020
number                                      4
volume                                    135
URL       https://doi.org/10.1093/qje/qjaa021
Name: 2, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjaa020_wo_cover.pdf
year                             

found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjaa001_wo_cover.pdf
year                                     2020
number                                      2
volume                                    135
URL       https://doi.org/10.1093/qje/qjaa001
Name: 29, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjz036_wo_cover.pdf
year                                    2020
number                                     1
volume                                   135
URL       https://doi.org/10.1093/qje/qjz036
Name: 30, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjz035_wo_cover.pdf
year                                    2020
number                                     1
volume                                   135
URL       https://doi.org/10.1093/qje/qjz035
Name: 31, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjz034_wo_cover.pdf
year                               

found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjz006_wo_cover.pdf
year                                    2019
number                                     3
volume                                   134
URL       https://doi.org/10.1093/qje/qjz006
Name: 58, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjz005_wo_cover.pdf
year                                    2019
number                                     3
volume                                   134
URL       https://doi.org/10.1093/qje/qjz005
Name: 59, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjz004_wo_cover.pdf
year                                    2019
number                                     2
volume                                   134
URL       https://doi.org/10.1093/qje/qjz004
Name: 61, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjz003_wo_cover.pdf
year                                    

found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjy010_wo_cover.pdf
year                                    2018
number                                     4
volume                                   133
URL       https://doi.org/10.1093/qje/qjy010
Name: 88, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjy009_wo_cover.pdf
year                                    2018
number                                     4
volume                                   133
URL       https://doi.org/10.1093/qje/qjy009
Name: 89, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjy008_wo_cover.pdf
year                                    2018
number                                     4
volume                                   133
URL       https://doi.org/10.1093/qje/qjy008
Name: 90, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjy007_wo_cover.pdf
year                                    

found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjx029_wo_cover.pdf
year                                    2018
number                                     1
volume                                   133
URL       https://doi.org/10.1093/qje/qjx029
Name: 117, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjx028_wo_cover.pdf
year                                    2018
number                                     1
volume                                   133
URL       https://doi.org/10.1093/qje/qjx028
Name: 118, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjx027_wo_cover.pdf
year                                    2018
number                                     1
volume                                   133
URL       https://doi.org/10.1093/qje/qjx027
Name: 119, dtype: object
found
/Users/sijiawu/Work/Refs Danae/Thesis/Data/PDFs/QJE/wo_cover/qjx026_wo_cover.pdf
year                                 

In [ ]:
dict={}

In [ ]:
t0=time.time()

for i in Merged[(Merged['year']>=1977) & (Merged['year']<=1979) & (Merged['content_type']!='MISC') & (Merged['content_type']!='Review')].index:
    if Merged.iloc[i]['Jstor_authors'] is not NaN: 
        if "Suggested by" not in Merged.iloc[i]['Jstor_authors']:
            authors=str(Merged.iloc[i]['Jstor_authors']).replace(' and ',', ').replace("  ",' ').split(',')
            filepath=path+'\\'+Merged.iloc[i]['stable_url'].split('/')[-1]+'.pdf'
            if os.path.exists(filepath)==True:
                print(filepath)
                print(Merged.iloc[i]['year'])
                print(authors)
                ignore='(('+Merged.iloc[i]['Jstor_title'].upper()+'){e<=3}(\n| \d| )|^(THIS CONTENT DOWNLOADED){e<=3}|^(QUARTERLY JOURNAL OF ECONOMICS){e<=3}\n|^\d)'
                affiliations=generate_pngs(filepath, mat, path, 38, '(^|\n)(REFERENCES){e<=3}(\n| )', ignore)                
                print(affiliations)
                dict[Merged.iloc[i]['stable_url'].split('/')[-1]]={'affiliations': affiliations, 
                                                                   'content_type': Merged.iloc[i]['content_type'], 
                                                                   'authors': authors, 
                                                                   'stable_url': Merged.iloc[i]['stable_url']}
            else:
                dict[Merged.iloc[i]['stable_url'].split('/')[-1]]='PDF not available, download at '+ Merged.iloc[i]['stable_url']
t1=time.time()
total=t1-t0
print(total)
print(i)

In [63]:
import json
with open(base_path+'\QJE_refs_output_2018_2020.json','w') as fp:
    json.dump(dict_ref, fp)

In [ ]:
count=0
for l in dict.keys():
    if 'found' in dict[l]['affiliations'].keys():
        count+=1
    else:
        print(l)
print(len(dict))
print(count)